In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import imageio
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
mpl.rcParams['figure.dpi'] = 50

class Agent:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color  

def move_agents(agents, speed_map, width, height):
    interaction_distance = 5  
    for agent in agents:
        if np.isnan(speed_map[int(agent.y), int(agent.x)]):
            continue
        
        speed = speed_map[int(agent.y), int(agent.x)]
        angle = np.random.uniform(0, 2 * np.pi)
        dx = np.cos(angle) * speed
        dy = np.sin(angle) * speed

        new_x = agent.x + dx
        new_y = agent.y + dy

        if new_x < 0:
            agent.x = -new_x  
        elif new_x >= width:
            agent.x = 2*width - new_x - 1  
        else:
            agent.x = new_x

        if new_y < 0:
            agent.y = -new_y  
        elif new_y >= height:
            agent.y = 2*height - new_y - 1  
        else:
            agent.y = new_y
        
        for other in agents:
            if other != agent and np.hypot(agent.x - other.x, agent.y - other.y) < interaction_distance:
                if np.random.random() < 0.5:
                    agent.color = np.random.choice([agent.color, other.color])

                    
def capture_agents(agents, speed_map, filename):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(speed_map, cmap='bone', interpolation='nearest')
    for agent in agents:
        ax.plot(agent.x, agent.y, 'o', markersize=5, color=agent.color)
    ax.axis('off')
    plt.savefig(filename)
    plt.close()

    
buffer = 10  

speed_map_path = '/Users/braydennoh/Downloads/speed_map.tif'
output_folder = '/Users/braydennoh/Downloads/newdistrict/1'

with rasterio.open(speed_map_path) as src:
    speed_map = src.read(1)
    width, height = src.width, src.height

num_agents = 1000
agents = []
colors = ['red', 'blue'] * (num_agents // 2)
np.random.shuffle(colors)  

while len(agents) < num_agents:
    x = np.random.randint(buffer, width - buffer)
    y = np.random.randint(buffer, height - buffer)
    if not np.isnan(speed_map[y, x]):
        agents.append(Agent(x, y, colors.pop()))

num_iterations = 300
frames = []

for i in range(num_iterations):
    move_agents(agents, speed_map, width, height)
    filename = f"{output_folder}/frame_{i:03d}.png"
    capture_agents(agents, speed_map, filename)
    frames.append(filename)

gif_path = f"{output_folder}/simulation.gif"
with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
    for filename in frames:
        image = imageio.imread(filename)
        writer.append_data(image)

print("GIF created at:", gif_path)